# Process Analytics

## Learning Goals

### Process Historians

**Process historians** are widely used throughout the process industries. The core function is to record and provide access to **time series data** for **process tags**.

The tclab library includes the basic functions of a process historian implemented using `SQLite`, a widely used database for web applications and internet connected devices.

* An instance of the tclab historian requires a list of **data sources**. There is no limit on the number of data sources.
* The default data sources record T1, T2, Q1, and Q2
* Each data source consists of **(tag, function) pair**, where the **tag** is a string that uniquely identifies a time series, and **function**, with no arguments, returns the current value of the tag.
* The tclab historian always records time with the tag "Time"
* Data from the historian can be stored in .csv files, converted to Pandas data frames, or accessed directly from the historian data log.

What you should be able to do:

* Read a list of data sources, and describe what data is recorded.
* Add a tag and a function to a list of data sources.
* Save data from the historian to a .csv file, then load the saved data for analysis.

<hr>

**Study Question.** The following list of data sources has been specified for the tclab historian. `lab.T1` and `lab.T1` are current values of temperatures T1 and T2.

    `sources = [["T1", lambda: lab.T1], ["T_ave", lambda: (lab.T1 + lab.T1)/2], ["T2", lab.T2]]`
    
* How many sources are specifiied?
* One of these specification will result in an error. Which one. What is wrong? How would you fix it?
* Explain what is being done with the second data source.
* `lab.Q2` is a function defined so that executinig `lab.Q2()` returns a value. Specify a data source incorporating this function.
    
**Study Question.** Suppose you've written a novel control algorithm. The algorithm generates a floating point value that is denoted by the Python variable `y_est`. Given a list of data sources `sources`, add a source for this quantity with the tag "Y_EST".

**Study Question.** Suppose you have implemented a contrrol algorithm for a bioreactor application that utilizes a state space model. The state vector `x` has two elements representing temperature T and concentration C. Create a list of sources for the historian to record this data with tags "T" and "C".

<hr>

### State-Space Models

A linear state-space model is written in the form

\begin{align}
\frac{dx}{dt} & = A x + B_u u + B_d d \\
y & = C x
\end{align}

where $x$ is the **state vector** with $n$ elements, $u$ is a vector of manipulable inputs, $d$ is a vector of disturbance inputs, and $y$ is a vector of process measurements. 

* A state-space system is stable if all eigenvalues have negative real parts. 
* If any eigenvalue has a positive real part, then the system is unstable.
* Eigenvalues can be computed with `eigenvalues, _ = np.linalg.eig(A)`
* The negative inverse of an eigenvalue corresponds to a system time constant.

What you should be able to do:

* Convert systems of linear differential equations into state-space form, identifying all relevant vectors and matrices.
* For a given problem, determine if the vectors and matrices are of compatiable dimensions.
* Compute eigenvalues and determine stability.
* Simulate the response of a state-space model to inputs and disturbances.

<hr>

**Study Question.** Suppose $x$ has five elements, $u$ has two elements, and $d$ has 1 element, and there is one process measurement. What are the dimensions of matrices $A$, $B_u$, $B_d$, and $C$?

**Study Question.** Someone has proposed a different model a temperature control device that includes heat transfer to the surround assembly. The model is written

\begin{align}
C_{p,1}\frac{dT_1}{dt} & = U_a(T_2 - T_1) + Q_1  \\
C_{p,2}\frac{dT_2}{dt} & = U_a(T_1 - T_2) + U_b(T_{amb} - T_2)
\end{align}

where $Q_1$ is the manipulable input, $T_2$ is measured, and $T_{amb}$ is an unmeasured external disturbance.

* Rewrite this model in state space form. Identify the components of $x$, $u$, $d$, and $y$. Write out matrices $A$, $B_u$, $B_d$, and $C$ specifying all of the coefficients.
* What are the units of $Q_1$ in this model?

<hr>

### State Estimation

State estimation is a form of real-time process analytics employed in control systems. State estimation combines process models with real-time process measurements to provide estimates of process variables that may not be directly measureable.

At each time step $t_k$ there are two calculations to perform:

* **Model Prediction:** Use the model to update the state to the next time step, i.e., $\hat{x}_{k-1} \rightarrow \hat{x}_{k}^{pred}$ with the equation

\begin{align}
\hat{x}_k^{pred} & = \hat{x}_{k-1} + (t_k - t_{k-1}) ( A \hat{x}_{k-1} + B_u u_{k-1} + B_d \hat{d}_{k-1}) \\
\hat{y}_k^{pred} & = C \hat{x}_k^{pred}
\end{align}

* **Measurement Correction:** Use measurement $y_k$ to update $\hat{x}_{k}^{pred} \rightarrow \hat{x}_{k}$ with the equation

$$\hat{x}_{k} = \hat{x}_{k}^{pred} - (t_k - t_{k-1})L (\hat{y}_{k}^{pred} - y_k)$$ 

$L$ is a matrix of observer gains.

The estimation error is given by $e = \hat{x} - {x}$ which satisfies the differential equations

\begin{align}
\frac{de}{dt} & = (A - LC) e + B_d (\hat{d}-d)
\end{align}

where $d$ is the unmeasured disturbance, and $\hat{d}$ is an apriori estimate of $d$. If the estimate is accurate, then $d = \hat{d}$.

<hr>

**Study Question.** For a system with 2 states and 1 measurement, what is the dimension of the matrix $L$? 

**Study Question.** For the same system, what is the dimension of the product $LC$?

**Study Question.** Suppose you are given numerical values for $A$, $L$, and $C$, how could you determine if the observer will be stable?

**Study Question.** The following charts test the performance of two different observers for the same system consisting of a single heater/sensor assembly. T1 is the measured temperature, Ts is the estimated sensor temperature, Th is the estimated heater temperature.

![](./figures/state_estimation_1.png)

![](./figures/state_estimation_2.png)

* Which of these is showing the useful result? Be sure you can explain why one is acceptable and one is not.
* For the one that is not performing, what has gone wrong? How you propose to fix the problem?
* Data for A, L, and C are available. What calculation would you perform to verify your analysis?  What calculation would you perform to remedy the problem? Be sure you can write out the code (no more than five lines required) to do these calculations.

<hr>